# A Simple Feed Forward Neural Network from Scratch - Part 1

© 2023-2025 by [Damir Cavar](http://damir.cavar.me/)


**Prerequisites:**

In [ ]:
!pip install -U numpy
!pip install -U nltk
!pip install -U scipy

## Introduction

In [1]:
import numpy as np
from scipy.special import softmax

In [2]:
x = np.array([2, 3, 4, 5, 6, 7])
num_features = 5
W = np.random.rand(len(x), num_features)
b = np.random.rand(num_features)

In [7]:
x_reshaped = x.reshape(len(x), 1)
print(x_reshaped.T)
print(x_reshaped)


[[2 3 4 5 6 7]]
[[2]
 [3]
 [4]
 [5]
 [6]
 [7]]


In [8]:
W

array([[0.8619727 , 0.58781623, 0.08192724, 0.32024874, 0.37586709],
       [0.53599043, 0.65137649, 0.26528275, 0.95910965, 0.25807033],
       [0.59982028, 0.05620878, 0.39362914, 0.35532813, 0.46986993],
       [0.91292256, 0.80483918, 0.86397291, 0.685994  , 0.34818464],
       [0.53730966, 0.10045434, 0.69879575, 0.67768112, 0.96681444],
       [0.03872422, 0.76156201, 0.75984532, 0.6103069 , 0.43399413]])

In [9]:
b

array([0.98591156, 0.20060554, 0.71763632, 0.70060456, 0.07488563])

In [10]:
z = x_reshaped.T @ W + b

In [11]:
print(z)

[[14.7766497  13.51305862 17.08341187 17.40794855 14.06007926]]


In [ ]:
def relu(x):
    return np.maximum(0.0,x)

In [14]:
a = relu(z)
a

array([[14.7766497 , 13.51305862, 17.08341187, 17.40794855, 14.06007926]])

In [15]:
U = np.random.rand(len(a[0]), 1)

In [16]:
U

array([[0.62690381],
       [0.35202677],
       [0.69657002],
       [0.70607078],
       [0.27014452]])

In [17]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

In [18]:
print(softmax(np.array([[3.0, 1.0, 0.2]])))
print(sum(softmax(np.array([[30.0, 10.0, 2.0]]))[0]))

[[0.8360188  0.11314284 0.05083836]]
0.9999999999999999


In [21]:
z2 = a @ U
print(z2)
sigmoid(z2[0][0])
print(sigmoid(z2.item()))

[[42.0097863]]
1.0


## Sentiment Analysis Example

In [22]:
import os
import csv
from nltk.tokenize import word_tokenize
from collections import Counter
import math
import numpy as np

In [2]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

Load the Reviews corpus:

In [23]:
experiment_data = []
with open(os.path.join('.', 'data', 'reviews.csv'), newline='') as csvfile:
    datareader = csv.reader(csvfile, delimiter=',', quotechar='"')
    header = next(datareader)
    for row in datareader:
        if len(row) == 2:
            experiment_data.append( [row[0].strip(), int(row[1].strip())] )

In [24]:
print(experiment_data[:2])
print("Text:\n", experiment_data[0][0])
print("Value:\n", experiment_data[0][1])
print("Number of records:", len(experiment_data))

[["Once again Mr. Costner has dragged out a movie for far longer than necessary. Aside from the terrific sea rescue sequences, of which there are very few I just did not care about any of the characters. Most of us have ghosts in the closet, and Costner's character are realized early on, and then forgotten until much later, by which time I did not care. The character we should really care about is a very cocky, overconfident Ashton Kutcher. The problem is he comes off as kid who thinks he's better than anyone else around him and shows no signs of a cluttered closet. His only obstacle appears to be winning over Costner. Finally when we are well past the half way point of this stinker, Costner tells us all about Kutcher's ghosts. We are told why Kutcher is driven to be the best with no prior inkling or foreshadowing. No magic here, it was all I could do to keep from turning it off an hour in.", 0], ["This is an example of why the majority of action films are the same. Generic and boring,

Load the Vader lexicon for lexical sentiment analysis:

In [25]:
sentiment_dictionary = {}
with open(os.path.join('.', 'data', 'vader_lexicon.txt'), mode='r', encoding='utf-8') as ifile:
    lines = ifile.readlines()
    sentiment_dictionary = { y[0]: y[1] for y in [ x.split('\t') for x in lines ] if len(y) == 4 }

In [26]:
print(sentiment_dictionary["adventurer"])

1.2


Define the set of 1st and 2nd person pronouns:

In [27]:
pronouns = {"i", "me", "my", "mine", "you", "yours", "yourself", "myself", "we", "us", "our", "ours", "ourselves"}

The vectorization function for the review texts is:

In [28]:
def vectorizer(text: str) -> list:
    tokens = word_tokenize(text.lower())
    scores = [ float(sentiment_dictionary.get(t, 0.0)) for t in tokens ]
    positive = len([ s for s in scores if s > 0 ])
    negative = len([ s for s in scores if s < 0 ])
    if "no" in tokens:
        no_present = 1
    else:
        no_present = 0
    counts = Counter(tokens)
    pronoun_count = 0
    for x in set(counts.keys()).intersection(pronouns):
        pronoun_count += counts[x]
    if "!" in tokens:
        exclamation = 1
    else:
        exclamation = 0
    return [positive, negative, no_present, pronoun_count, exclamation, math.log(len(tokens))]

Generate the vectors for all the corpus texts:

In [29]:
x = [ np.array(vectorizer(e[0])) for e in experiment_data ]

In [30]:
print(len(x), len(x[0]))
print(x[1], "\n", experiment_data[1], "\n", experiment_data[1][0], "\n", experiment_data[1][1])
y = np.array([ float(e[1]) for e in experiment_data ])
print(y)
print(len(y))

50000 6
[13.          8.          0.          3.          0.          5.58349631] 
 ["This is an example of why the majority of action films are the same. Generic and boring, there's really nothing worth watching here. A complete waste of the then barely-tapped talents of Ice-T and Ice Cube, who've each proven many times over that they are capable of acting, and acting well. Don't bother with this one, go see New Jack City, Ricochet or watch New York Undercover for Ice-T, or Boyz n the Hood, Higher Learning or Friday for Ice Cube and see the real deal. Ice-T's horribly cliched dialogue alone makes this film grate at the teeth, and I'm still wondering what the heck Bill Paxton was doing in this film? And why the heck does he always play the exact same character? From Aliens onward, every film I've seen with Bill Paxton has him playing the exact same irritating character, and at least in Aliens his character died, which made it somewhat gratifying... Overall, this is second-rate action t

In [10]:
X = np.array(x)
print(X)

[[ 9.          9.          1.          8.          0.          5.23110862]
 [13.          8.          0.          3.          0.          5.58349631]
 [ 6.         14.          1.          5.          0.          5.49306144]
 ...
 [22.         18.          0.         12.          0.          6.14632926]
 [11.          0.          0.         12.          0.          5.32300998]
 [11.          7.          1.          1.          1.          5.27299956]]


Define the RelU activation function:

In [179]:
def relu(x):
    return np.maximum(0.0, x)

Initialize the variables and weights for the network:

In [219]:
print(x[0])
num_features = len(x[0])
num_rows = 3
W = np.random.rand(num_rows, num_features)
print("W:\n", W)
b = np.random.rand(num_rows)
print("b:\n", b)
print("W+b:\n", relu(W @ x[0] + b) )

[9.         9.         1.         8.         0.         5.23110862]
W:
 [[0.15712481 0.71373242 0.43670908 0.92744404 0.8657041  0.67280756]
 [0.04211492 0.48538567 0.70691137 0.13823402 0.06752036 0.31788857]
 [0.33065637 0.17537358 0.15559781 0.30787449 0.9475915  0.78308026]]
b:
 [0.51426813 0.22715494 0.78433198]
W+b:
 [19.72777397  8.45035339 12.05357319]


The output layer receives a *num-rows*-dimentional vector and generates one output z-score. Initialize the output layer of the network:

In [220]:
U = np.random.rand(1, num_rows)
print("U:\n", U)
bu = np.random.rand(1)
print("bu:\n", bu)
test = W @ x[0] + b
print(test)
print(sigmoid(U @ test + bu))

U:
 [[0.34349942 0.37909158 0.55288836]]
bu:
 [0.6822026]
[19.72777397  8.45035339 12.05357319]
[0.99999997]


The training data set in the following code is the entire data set. As an exercise use some function to randomly select a portion for training and another portion for testing. Use only the true randomly selected training data set for the following code. This code will run a complete inferencing cycle through the training data set, which is in this case the entire data set. Change the code and set up a real evaluation of training and testing, and computing of the F1-score.

In [221]:
results = []
for input_vector, truth in zip(x, y):
    z = (W @ input_vector) + b
    #print("z:\n", z)
    a = relu(z)
    #print("a:\n", a)
    c = sigmoid((U @ a) + bu)
    #print("c:\n", c)
    results.append( (truth, c) )

In [222]:
print("# of results:", len(results))

# of results: 50000


In [224]:
counting_true_negatives = 0
counting_true_positives = 0
counting_false_positives = 0
counting_false_negatives = 0
for res in results:
    # print(res[0], res[1][0])
    if res[0] == 1:
        if res[1][0] >= 0.5:
            counting_true_positives += 1
        else:
            counting_false_negatives += 1
    else:
        if res[1][0] < 0.5:
            counting_true_negatives += 1
        else:
            counting_false_positives += 1
print("True positives:", counting_true_positives)
print("True negatives:", counting_true_negatives)
print("False positives:", counting_false_positives)
print("False negatives:", counting_false_negatives)

True positives: 25000
True negatives: 0
False positives: 25000
False negatives: 0


Continuation in *SimpleFeedForwardNetwork_2*.

© 2025 by [Damir Cavar](https://damir.cavar.me/)